In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 125
import seaborn as sns
from Bio import SeqIO, Seq
import scipy.stats as st
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import glob, os, yaml, subprocess, itertools, sparse, pickle

who_variants = pd.read_csv("/n/data1/hms/dbmi/farhat/Sanjana/MIC_data/WHO_resistance_variants_all.csv")
pd.set_option('display.float_format', lambda x: '%.4f' % x)
samples_summary = pd.read_csv("../data/samples_summary.csv")
analysis_dir = '/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue'

In [157]:
drug = "Delamanid"
# final_analysis file with all significant variants for a drug
res_df = pd.read_csv(os.path.join(analysis_dir, drug, "final_analysis.csv"))
# df_phenos = pd.read_csv(os.path.join(analysis_dir, drug, "phenos_binary.csv"))

# df_genos = pd.read_csv(os.path.join(analysis_dir, drug, "genos.csv.gz"), compression="gzip")
# df_genos["orig_variant"] = df_genos["resolved_symbol"] + "_" + df_genos["variant_category"]

In [158]:
res_df.columns

Index(['orig_variant', 'coef', 'coef_LB', 'coef_UB', 'pval', 'BH_pval',
       'Bonferroni_pval', 'confidence_WHO_2021', 'Odds_Ratio', 'OR_LB',
       'OR_UB', 'Tier', 'Phenos', 'unpooled', 'synonymous', 'Num_Isolates',
       'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV', 'Sens', 'Spec', 'LR+',
       'LR-', 'PPV_LB', 'PPV_UB', 'Sens_LB', 'Sens_UB', 'Spec_LB', 'Spec_UB',
       'LR+_LB', 'LR+_UB', 'LR-_LB', 'LR-_UB', 'predicted_effect'],
      dtype='object')

In [159]:
res_df

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,confidence_WHO_2021,Odds_Ratio,OR_LB,...,PPV_UB,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,predicted_effect
0,ddn_lof,0.2951,0.1698,0.3745,0.0000,0.0000,0.0000,NaN,1.3432,1.1851,...,0.9444,0.0443,0.1083,0.9991,0.9999,68.8174,770.5864,0.8922,0.9561,lof
1,fgd1_lof,0.1563,-0.0000,0.2080,0.0041,0.0332,0.2990,NaN,1.1692,1.0000,...,1.0000,0.0000,0.0218,1.0000,1.0000,inf,inf,0.9782,1.0000,lof
2,ddn_p.Val127Phe,0.1547,-0.0273,0.2292,0.0025,0.0299,0.1792,NaN,1.1673,0.9731,...,1.0000,0.0000,0.0264,0.9996,1.0000,0.0000,inf,0.9738,1.0002,missense_variant
3,ddn_p.Leu49Pro,0.1377,0.1170,0.1530,0.0000,0.0000,0.0000,2) Assoc w R - Interim,1.1476,1.1241,...,1.0000,0.0201,0.0717,0.9996,1.0000,78.2850,inf,0.9285,0.9801,missense_variant
4,fbiB_p.Lys448Arg,0.1318,-0.0342,0.2785,0.0537,0.1189,1.0000,3) Uncertain significance,1.1409,0.9664,...,0.1617,0.0456,0.1083,0.9854,0.9892,3.5155,8.5974,0.9041,0.9667,missense_variant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,fbiB_p.Leu447Arg,-0.0649,-0.0788,-0.0504,0.0000,0.0000,0.0000,3) Uncertain significance,0.9371,0.9242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missense_variant
954,fgd1_c.465C>T,-0.0783,-0.0901,-0.0657,0.0000,0.0000,0.0000,NaN,0.9247,0.9139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,synonymous_variant
955,fbiA_p.Arg304Gln,-0.0799,-0.1325,0.0000,0.0480,0.1129,1.0000,3) Uncertain significance,0.9232,0.8759,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missense_variant
956,ndh_lof,-0.1093,-0.1496,-0.0602,0.0000,0.0000,0.0001,NaN,0.8964,0.8610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lof


In [160]:
res_df.loc[pd.isnull(res_df["predicted_effect"])]

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,confidence_WHO_2021,Odds_Ratio,OR_LB,...,PPV_UB,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,predicted_effect
11,PC4,0.1016,-0.0728,0.2469,0.1077,0.1228,1.0000,NaN,1.1069,0.9298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,PC2,0.0605,-0.1190,0.2391,0.2571,0.2844,1.0000,NaN,1.0624,0.8878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,PC3,0.0300,-0.1110,0.1728,0.3431,0.3738,1.0000,NaN,1.0305,0.8950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,PC1,-0.0039,-0.1845,0.1639,0.4829,0.5109,1.0000,NaN,0.9961,0.8316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
957,PC0,-0.2291,-0.4017,-0.0775,0.0026,0.0274,0.1919,NaN,0.7953,0.6692,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
res_df.columns

Index(['orig_variant', 'coef', 'coef_LB', 'coef_UB', 'pval', 'BH_pval',
       'Bonferroni_pval', 'confidence_WHO_2021', 'Odds_Ratio', 'OR_LB',
       'OR_UB', 'Tier', 'Phenos', 'unpooled', 'synonymous', 'Num_Isolates',
       'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV', 'Sens', 'Spec', 'LR+',
       'LR-', 'PPV_LB', 'PPV_UB', 'Sens_LB', 'Sens_UB', 'Spec_LB', 'Spec_UB',
       'LR+_LB', 'LR+_UB', 'LR-_LB', 'LR-_UB', 'predicted_effect'],
      dtype='object')

# Combined Analysis Files

In [92]:
def final_processing(drug):
    '''
    Functions for processing outputs before sending to everyone else.
    
    1. Remove principal components (will describe them separately)
    2. Add LOF to the predicted_effect column for pooled LOF mutations
    3. Remove genome_index column (should actually do that earlier, but will fix later)
    4. Remove the logistic regression coefficient columns (they will prefer to work with odds ratios)
    5. Any other column renaming or dropping for clarity
    '''
    
    analysis_df = pd.read_csv(os.path.join(analysis_dir, drug, "final_analysis.csv"))
    analysis_df.rename(columns={"orig_variant": "mutation"}, inplace=True)
    
    # remove logReg coefficients. Keep only odds ratios. Remove the other two columns, which were present mainly for me to see
    # if we were picking up many mutations that were in the 2021 mutation catalog
    del analysis_df["confidence_WHO_2021"]
    analysis_df = analysis_df[analysis_df.columns[~analysis_df.columns.str.contains("coef")]]
    
    # remove significant principal components and replace the NaNs in the predicted effect column for the gene loss of functions
    analysis_df = analysis_df.loc[~analysis_df["mutation"].str.contains("PC")]
    
    # predicted effect should not be NaN for anything. position is NaN only for the pooled LOF mutations
    assert len(analysis_df.loc[pd.isnull(analysis_df["predicted_effect"])]) == 0
    
    # reorder columns
    analysis_df = analysis_df[['mutation', 'predicted_effect', 'Odds_Ratio', 'OR_LB', 'OR_UB', 'pval', 'BH_pval', 'Bonferroni_pval',
       'Num_Isolates', 'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'Sens_LB', 'Sens', 'Sens_UB', 'Spec_LB', 'Spec', 'Spec_UB', 'PPV', 'PPV_LB', 'PPV_UB',
       'LR+_LB', 'LR+', 'LR+_UB', 'LR-_LB', 'LR-', 'LR-_UB', 'Tier', 'Phenos', 'unpooled']]
            
    return analysis_df

In [93]:
test = final_processing("Delamanid")

In [94]:
test

,mutation,predicted_effect,Odds_Ratio,OR_LB,OR_UB,pval,BH_pval,Bonferroni_pval,Num_Isolates,Total_Isolates,...,PPV_UB,LR+_LB,LR+,LR+_UB,LR-_LB,LR-,LR-_UB,Tier,Phenos,unpooled
0,ddn_lof,lof,1.3432,1.1851,1.4543,0.0000,0.0000,0.0000,24.0000,11803.0000,...,0.9500,73.3231,174.1817,852.3245,0.8908,0.9250,0.9559,1,WHO,0
1,fgd1_lof,lof,1.1692,1.0000,1.2312,0.0041,0.0332,0.2990,2.0000,11803.0000,...,1.0000,inf,inf,inf,0.9806,0.9921,1.0000,1,WHO,0
2,ddn_p.Val127Phe,missense_variant,1.1673,0.9731,1.2576,0.0025,0.0299,0.1792,5.0000,11803.0000,...,1.0000,0.0000,68.7560,inf,0.9739,0.9883,1.0002,1,WHO,0
3,ddn_p.Leu49Pro,missense_variant,1.1476,1.1241,1.1654,0.0000,0.0000,0.0000,13.0000,11803.0000,...,1.0000,75.1657,252.1052,inf,0.9278,0.9565,0.9792,2,ALL,1
4,fbiB_p.Lys448Arg,missense_variant,1.1409,0.9664,1.3211,0.0537,0.1189,1.0000,165.0000,11803.0000,...,0.1667,3.2246,5.9651,9.1374,0.9028,0.9364,0.9713,1,WHO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,fbiC_c.967C>T,synonymous_variant,0.9374,0.9270,0.9490,0.0000,0.0000,0.0000,35.0000,11803.0000,...,NaN,NaN,0.0000,NaN,NaN,1.0030,NaN,2,ALL,0
953,fbiB_p.Leu447Arg,missense_variant,0.9371,0.9242,0.9509,0.0000,0.0000,0.0000,33.0000,11803.0000,...,NaN,NaN,0.0000,NaN,NaN,1.0029,NaN,2,ALL,1
954,fgd1_c.465C>T,synonymous_variant,0.9247,0.9139,0.9364,0.0000,0.0000,0.0000,71.0000,11803.0000,...,NaN,NaN,0.0000,NaN,NaN,1.0062,NaN,2,ALL,0
955,fbiA_p.Arg304Gln,missense_variant,0.9232,0.8759,1.0000,0.0480,0.1129,1.0000,1.0000,11803.0000,...,NaN,NaN,0.0000,NaN,NaN,1.0001,NaN,1,WHO,0


In [95]:
drug = "Delamanid"
core_model_path = os.path.join(analysis_dir, drug, "tiers=1/phenos=WHO/dropAF_noSyn")

core_analysis = pd.read_csv(os.path.join(core_model_path, "model_analysis.csv"))
core_analysis["Tier"] = 1
core_analysis["Phenos"] = "WHO"
core_analysis["unpooled"] = 0
core_analysis["synonymous"] = 0

# create dictionary of the additional model analyses
add_analyses = {}

for tier in os.listdir(os.path.join(analysis_dir, drug)):
    tiers_path = os.path.join(analysis_dir, drug, tier)
    if os.path.isdir(tiers_path):
        for pheno in os.listdir(tiers_path):
            phenos_path = os.path.join(analysis_dir, drug, tier, pheno)
            for model in os.listdir(phenos_path):
                model_path = os.path.join(analysis_dir, drug, tier, pheno, model)

                analysis_fName = os.path.join(model_path, "model_analysis.csv")
                if os.path.isfile(analysis_fName):
                    
                    # only include models where HETs were dropped because we can not compute univariate stats for continuous AFs
                    if model_path != core_model_path:
                        if ("encodeAF" not in model_path):
                            add_analysis = pd.read_csv(analysis_fName)
                            add_path = model_path.split(os.path.join(analysis_dir, drug))[1].strip("/").replace("dropAF_", "")

                            add_analysis["Tier"] = [2 if "+2" in add_path else 1][0]
                            add_analysis["Phenos"] = ["ALL" if "ALL" in add_path else "WHO"][0]
                            add_analysis["unpooled"] = int("unpooled" in add_path)
                            add_analysis["synonymous"] = int("withSyn" in add_path)

                            # keep all significant variants at an FDR threshold of 0.01
                            add_analyses[add_path] = add_analysis
                        else:
                            print(f"Skipping {model_path} because this model encodes Hets with AFs.")
                else:
                    print(f"No model analysis file in {analysis_fName}")
    else:
        print(f"Skipping {tiers_path} because it is not a directory")
      
keys_lst = list(add_analyses.keys())

# merge the core analysis dataframe with the first additional dataframe
merged_df = pd.concat([core_analysis, add_analyses[keys_lst[0]]], axis=0).drop_duplicates("orig_variant", keep="first")

# merge the merged dataframe with the remaining additional analyses, dropping duplicates as before
for key in keys_lst[1:]:
    merged_df = pd.concat([merged_df, add_analyses[key]], axis=0).drop_duplicates("orig_variant", keep="first")

del merged_df["genome_index"]

Skipping /n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Delamanid/final_analysis.csv because it is not a directory
Skipping /n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Delamanid/tiers=1+2/phenos=ALL/encodeAF_noSyn because this model encodes Hets with AFs.
Skipping /n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Delamanid/tiers=1+2/phenos=WHO/encodeAF_noSyn because this model encodes Hets with AFs.
Skipping /n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Delamanid/genos.csv.gz because it is not a directory
Skipping /n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Delamanid/phenos_binary.csv because it is not a directory
Skipping /n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Delamanid/tiers=1/phenos=ALL/encodeAF_noSyn because this model encodes Hets with AFs.
Skipping /n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Delamanid/tiers=1/phenos=WHO/encodeAF_noSyn because this model encodes Hets with AFs.


In [99]:
merged_df

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,confidence_WHO_2021,Odds_Ratio,OR_LB,OR_UB,Tier,Phenos,unpooled,synonymous
0,ddn_lof,0.2951,0.1698,0.3745,0.0000,0.0000,0.0000,NaN,1.3432,1.1851,1.4543,1,WHO,0,0
1,fgd1_lof,0.1563,-0.0000,0.2080,0.0041,0.0332,0.2990,NaN,1.1692,1.0000,1.2312,1,WHO,0,0
2,ddn_p.Val127Phe,0.1547,-0.0273,0.2292,0.0025,0.0299,0.1792,NaN,1.1673,0.9731,1.2576,1,WHO,0,0
3,fbiB_p.Lys448Arg,0.1318,-0.0342,0.2785,0.0537,0.1189,1.0000,3) Uncertain significance,1.1409,0.9664,1.3211,1,WHO,0,0
4,fbiC_lof,0.1286,-0.0551,0.2062,0.0380,0.0926,1.0000,NaN,1.1372,0.9464,1.2290,1,WHO,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,fbiC_c.24G>A,-0.0458,-0.0544,-0.0340,0.0000,0.0000,0.0000,NaN,0.9552,0.9471,0.9666,2,ALL,0,1
891,fbiC_c.396C>T,-0.0513,-0.0610,-0.0387,0.0000,0.0000,0.0000,NaN,0.9500,0.9408,0.9620,2,ALL,0,1
896,fgd1_c.774G>A,-0.0641,-0.0745,-0.0505,0.0000,0.0000,0.0000,NaN,0.9379,0.9282,0.9507,2,ALL,0,1
897,fbiC_c.967C>T,-0.0646,-0.0758,-0.0524,0.0000,0.0000,0.0000,NaN,0.9374,0.9270,0.9490,2,ALL,0,1


In [97]:
def compute_predictive_values(combined_df, return_stats=[]):
    '''
    Compute positive predictive value. 
    Compute sensitivity, specificity, and positive and negative likelihood ratios. 
    
    PPV = true_positive / all_positive. NPV = true_negative / all_negative
    Sens = true_positive / (true_positive + false_negative)
    Spec = true_negative / (true_negative + false_positive)
    
    Also return the number of isolates with each variant = all_positive
    
    Positive LR = sens / (1 – spec)
    Negative LR = (1 – sens) / spec

    '''
    # make a copy to keep sample_id in one dataframe
    melted = combined_df.melt(id_vars=["sample_id", "phenotype"])
    melted_2 = melted.copy()
    del melted_2["sample_id"]
    
    # get counts of isolates grouped by phenotype and variant -- so how many isolates have a variant and have a phenotype (all 4 possibilities)
    grouped_df = pd.DataFrame(melted_2.groupby(["phenotype", "variable"]).value_counts()).reset_index()
    grouped_df = grouped_df.rename(columns={"variable": "orig_variant", "value": "variant", 0:"count"})
    
    # dataframes of the counts of the 4 values
    true_pos_df = grouped_df.query("variant == 1 & phenotype == 1").rename(columns={"count": "TP"})
    false_pos_df = grouped_df.query("variant == 1 & phenotype == 0").rename(columns={"count": "FP"})
    true_neg_df = grouped_df.query("variant == 0 & phenotype == 0").rename(columns={"count": "TN"})
    false_neg_df = grouped_df.query("variant == 0 & phenotype == 1").rename(columns={"count": "FN"})

    assert len(true_pos_df) + len(false_pos_df) + len(true_neg_df) + len(false_neg_df) == len(grouped_df)
    
    # combine the 4 dataframes into a single dataframe (concatenating on axis = 1)
    final = true_pos_df[["orig_variant", "TP"]].merge(
            false_pos_df[["orig_variant", "FP"]], on="orig_variant", how="outer").merge(
            true_neg_df[["orig_variant", "TN"]], on="orig_variant", how="outer").merge(
            false_neg_df[["orig_variant", "FN"]], on="orig_variant", how="outer").fillna(0)

    assert len(final) == len(melted["variable"].unique())
    assert len(final) == len(final.drop_duplicates("orig_variant"))
        
    final["Num_Isolates"] = final["TP"] + final["FP"]
    final["Total_Isolates"] = final["TP"] + final["FP"] + final["TN"] + final["FN"]
    final["PPV"] = final["TP"] / (final["TP"] + final["FP"])
    final["Sens"] = final["TP"] / (final["TP"] + final["FN"])
    final["Spec"] = final["TN"] / (final["TN"] + final["FP"])
    final["LR+"] = final["Sens"] / (1 - final["Spec"])
    final["LR-"] = (1 - final["Sens"]) / final["Spec"]
    #final["NPV"] = final["TN"] / (final["TN"] + final["FN"])
    
    if len(return_stats) == 0:
        return final[["orig_variant", "Num_Isolates", "Total_Isolates", "TP", "FP", "TN", "FN", "PPV", "Sens", "Spec", "LR+", "LR-"]]
    else:
        return final[return_stats]

In [100]:
df_phenos = pd.read_csv(os.path.join(analysis_dir, drug, "phenos_binary.csv"))
df_genos = pd.read_csv(os.path.join(analysis_dir, drug, "genos.csv.gz"), compression="gzip")
df_genos["orig_variant"] = df_genos["resolved_symbol"] + "_" + df_genos["variant_category"]
df_copy = df_genos.copy()

# pool LOF and inframe mutations
df_copy.loc[df_copy["predicted_effect"].isin(["frameshift", "start_lost", "stop_gained", "feature_ablation"]), ["variant_category", "position"]] = ["lof", np.nan]
df_copy.loc[df_copy["predicted_effect"].isin(["inframe_insertion", "inframe_deletion"]), ["variant_category", "position"]] = ["inframe", np.nan]

# update the orig_variant column using the new variant categories (lof and inframe) and combine the pooled and unpooled variants
df_copy["orig_variant"] = df_copy["resolved_symbol"] + "_" + df_copy["variant_category"]
df_pooled = df_copy.query(f"variant_category in ['lof', 'inframe']").sort_values(by=["variant_binary_status", "variant_allele_frequency"], ascending=False, na_position="last").drop_duplicates(subset=["sample_id", "orig_variant"], keep="first")
del df_copy
df_genos_full = pd.concat([df_genos, df_pooled], axis=0)
del df_genos
del df_pooled

# keep only variants that are in the final_analysis dataframe and drop NaNs (NaNs = either isolate didn't pass QC or it's a Het) 
# We can't process Hets here because they need to be binary to have univariate statistics
df_genos_full = df_genos_full.loc[(df_genos_full["orig_variant"].isin(merged_df["orig_variant"].values))].dropna(subset="variant_binary_status")

# check that the only variants that are in res_df but not in df_genos_full are the principal components
if sum(~pd.Series(list(set(merged_df["orig_variant"]) - set(df_genos_full["orig_variant"]))).str.contains("PC")) > 0:
    raise ValueError("Variants are missing from df_genos_full!")

combined = df_genos_full.pivot(index="sample_id", columns="orig_variant", values="variant_binary_status")

# predicted effect annotations for later
annotated_genos = df_genos_full.query("variant_category not in ['lof', 'inframe']").drop_duplicates(["orig_variant", "predicted_effect"])[["orig_variant", "predicted_effect"]]
del df_genos_full
combined = combined.merge(df_phenos[["sample_id", "phenotype"]], left_index=True, right_on="sample_id").reset_index(drop=True)

# get dataframe of predictive values for the non-zero coefficients and add them to the results dataframe
full_predict_values = compute_predictive_values(combined)
merged_df = merged_df.merge(full_predict_values, on="orig_variant", how="outer")

/tmp/ipykernel_6135/3506956419.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_genos = pd.read_csv(os.path.join(analysis_dir, drug, "genos.csv.gz"), compression="gzip")


In [111]:
merged_df.columns

Index(['orig_variant', 'coef', 'coef_LB', 'coef_UB', 'pval', 'BH_pval',
       'Bonferroni_pval', 'confidence_WHO_2021', 'Odds_Ratio', 'OR_LB',
       'OR_UB', 'Tier', 'Phenos', 'unpooled', 'synonymous', 'Num_Isolates',
       'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV', 'Sens', 'Spec', 'LR+',
       'LR-'],
      dtype='object')

In [109]:
keep_variants = list(merged_df.loc[(~merged_df["orig_variant"].str.contains("PC")) & (merged_df["coef"] > 0)]["orig_variant"].values)

# Remake this dataframe with fewer features because only going to bootstrap stats for variants with positive coefficients.
# check that all samples were preserved. 
combined_small = combined[["sample_id", "phenotype"] + keep_variants]
assert len(combined_small) == len(combined)

bs_results = pd.DataFrame(columns = keep_variants)

In [112]:
bs_results

,ddn_lof,fgd1_lof,ddn_p.Val127Phe,fbiB_p.Lys448Arg,fbiC_lof,fbiA_lof,fgd1_inframe,fbiB_inframe,ddn_inframe,fbiA_p.Val47Ile,...,ndh_c.228G>A,fbiA_c.900G>A,fbiC_c.1392C>T,fbiB_c.1330T>C,fbiC_c.2235G>A,fbiC_c.2427C>G,fbiB_c.1335G>A,fbiB_c.90C>T,fbiA_c.337C>T,fbiA_c.15T>C


In [37]:
finished_drugs = []

for drug in os.listdir(analysis_dir):
    
    if os.path.isfile(os.path.join(analysis_dir, drug, "final_analysis.csv")):
        finished_drugs.append(drug)
        # drug_analyses.append(final_processing(drug))

In [ ]:
print(finished_drugs)

In [ ]:
# those that are actually done
finished_drugs = ['Pyrazinamide', 'Amikacin', 'Clofazimine', 'Linezolid', 'Moxifloxacin', 'Kanamycin', 'Bedaquiline', 'Capreomycin', 'Delamanid']

drug_analyses = {}
for drug in finished_drugs:
    drug_analyses[drug] = final_processing(drug)
    
# write results to an Excel file, where each sheet is named for a drug
with pd.ExcelWriter("Farhat_logReg_analysis.xlsx") as file:
   
    for key, val in drug_analyses.items():
        val.to_excel(file, sheet_name=key, index=False)

In [ ]:
drug_analyses[4].query("Tier==2&Phenos=='WHO'")

In [ ]:
drug_analyses[0].query("Tier==1&Phenos=='ALL'")

In [ ]:
drug_analyses[0].query("predicted_effect=='synonymous_variant' & ")

In [ ]:
moxi_df.loc[pd.isnull(moxi_df["position"])]

# Catalog-Based Method Comparison

In [ ]:
def get_logReg_summary(out_dir):
    
    model_summary = pd.read_csv(os.path.join(out_dir, "logReg_summary.csv"))
    model_analysis = pd.read_csv(os.path.join(out_dir, "model_analysis.csv"))
    
    summary_sens = model_summary.loc[0, "Sens"]
    summary_spec = model_summary.loc[0, "Spec"]
    
    #max_idx = [np.argmax(model_analysis["Sens"]), np.argmax(model_analysis["Spec"])]
    
    better_variant = model_analysis.query("Sens >= @summary_sens & Spec >= @summary_spec")
    
    if len(better_variant) == 0:
        print("No variants have comparable sensitivity AND specificity")
    else:
        print(better_variant)
    return model_summary, model_analysis
    
    #return model_summary, model_analysis.iloc[max_idx, :][["orig_variant", "coef", "Sens", "Spec", "accuracy", "balanced_accuracy"]]

In [ ]:
#out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Levofloxacin/tiers=1+2/phenos=ALL/dropAF_withSyn"
out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Pyrazinamide/tiers=1+2/phenos=WHO/dropAF_withSyn"

summary, analysis = get_logReg_summary(out_dir)
summary

In [ ]:
analysis.query("Sens > 0.9006")

In [ ]:
analysis.loc[analysis["orig_variant"].str.contains("PC")]

In [ ]:
analysis.query("Spec > 0.9575")

In [ ]:
out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Moxifloxacin/tiers=1+2/phenos=ALL/dropAF_withSyn"

get_logReg_summary(out_dir)

In [ ]:
df_pza = pd.read_csv("/n/data1/hms/dbmi/farhat/ye12/who/analysis/Pyrazinamide/tiers=1+2/phenos=WHO/dropAF_withSyn/phenos.csv")

# the missing ones might be M. cannettii, most similar to L6 based on the other lineage callers
lineages = pd.read_pickle("../data/combined_lineage_sample_IDs.pkl")
lineages["Lineage"] = lineages["Lineage"].fillna("6")
lineages["Lineage_1"] = lineages["Lineage_1"].fillna("6")

lineages = lineages[["Sample Name", "Sample ID", "Lineage_1"]]
lineages["Lineage"] = [str(val).split(".")[0] for val in lineages["Lineage_1"].values]
lineages.loc[lineages["Lineage"].str.contains("BOV"), "Lineage"] = "M. bovis"

assert len(lineages.loc[pd.isnull(lineages["Lineage"])]) == 0

########## KEEP ONLY ISOLATES WITH ALL 3 PIECES OF DATA ##########

# get only isolates with data for everyting: SNP matrix, in the model, and lineages
combined = lineages.merge(df_pza, left_on="Sample ID", right_on="sample_id")

In [ ]:
combined.groupby("Lineage")["phenotype"].mean().sort_values(ascending=False)

In [ ]:
combined.query("Lineage == 'M. bovis'").Lineage_1.unique()